Importando as Bibliotecas

In [23]:
import numpy as np
import random
import plotly.graph_objects as go
from scipy.special import gamma

#### Definindo Variáveis Iniciais

In [24]:
n = 10**4 # Número de vetores thetas
k = 10**1 # Número de bins
m = 3 # Dimensionalidade
x = np.random.randint(0, 11, m) # Vetor x
y = np.random.randint(0, 11, m) # Vetor y

alfa = x + y # Vetor Alfa = Vetor x + Vetor y
print(alfa)

[ 9 14 12]


#### Definindo Funções Principais

In [25]:
def f(theta, alfa, constante):
    log_produtorio = np.sum((alfa - 1) * np.log(theta), axis=1) # Para acelerar as operações, faz-se log do produtório (vira somatório)
    return np.exp(log_produtorio)/constante

def beta_multivariavel(alfa): # Beta(a + b) = Gamma(a)Gamma(b)/Gamma(a + b)
    log_produtorio = np.sum(np.log(gamma(alfa)))
    return np.exp(log_produtorio)/gamma(np.sum(alfa))

In [26]:
def gerador_dirichlet(alfa, n): # Essa função gera amostras dirichlet utilizando a função gamma: y_i = x_i/sum(x)
    # Gera amostras da distribuição gamma
    samples = np.random.gamma(alfa, 1, size=(n, m))
    
    # Normalize as amostras para que a soma de cada vetor seja 1
    thetas = samples / np.sum(samples, axis=1)[:, np.newaxis]

    return thetas

In [27]:
def criar_lista_v(lista_f_ord, n, k):
    # Define-se os pontos de corte v
    # - v_0 = 0; v_k = sup(f(theta))
    resto = n % k
    tamanho_bin = n // k

    # No entanto, quando n%k = c != 0, precisa-se adaptar o tamanho de c bins com um ponto a mais
    ajuste = np.concatenate((np.arange(1, resto + 1), np.array([resto] * (k - resto)))) # Array para ajustar o tamanho dos bins
    lista_intermed = np.arange(1, k + 1) * tamanho_bin + ajuste
    lista_v = np.concatenate(([0], lista_f_ord[lista_intermed - 1]))
    return lista_v, lista_intermed

#### Gerando Thetas e Aplicando $f(\theta)$

In [28]:
thetas = gerador_dirichlet(alfa, n) # Lista de Vetores theta

constante = beta_multivariavel(alfa)
lista_f = f(thetas, alfa, constante) # Aplicar f em cada vetor theta

#### Ordenando as Listas e Criando Cortes

In [29]:
lista_f_ord = np.sort(lista_f) # Ordena a lista de resultados
sup_f = lista_f_ord[-1] # Define o supremo de f(theta) como o valor máximo obtido

lista_v, lista_intermed = criar_lista_v(lista_f_ord, n, k)

#### Analisando Visualmente a relação magnitude x $f(\theta)$

In [30]:
lista_r = [np.linalg.norm(theta) for theta in thetas] # Encontra o módulo de cada vetor theta
lista_pontos = [(lista_r[i], lista_f[i]) for i in range(n)]

a = [p[0] for p in lista_pontos]
b = [p[1] for p in lista_pontos]

# Plotando o scatter plot
fig = go.Figure()

# Scatter plot de f(theta) x Módulo Theta
fig.add_trace(go.Scatter(x=a, y=b, mode='markers', marker=dict(color='blue', symbol='circle')))

# Adicionando as linhas de cada corte v
for v in lista_v:
    fig.add_shape(type="line", x0=0, y0=v, x1=5, y1=v, line=dict(color="red", dash="dash"))

# Layout
fig.update_layout(title='Distribuição da Função segundo Magnitude do Vetor e Cortes',
                  xaxis_title='Magnitude',
                  yaxis_title='f',
                  xaxis=dict(range=[min(a), max(a)]),
                  yaxis=dict(range=[min(b), max(b)]),
                  plot_bgcolor='rgba(0,0,0,0)') # Define a cor de fundo do plot

fig.show()

#### Interpolando os Dados e Expondo Gráficos

In [31]:
from scipy.interpolate import PchipInterpolator

In [32]:
fracao = np.concatenate((np.array([0]), lista_intermed))/n # Identifica a fração total de pontos de 0 até o ponto v

interp = PchipInterpolator(lista_v, fracao)
# Utiliza um interpolador PCHIP para gerar um polinômio de terceiro grau que aproxime W(v)
# Dessa forma, obtém-se U(v)

# Define os pontos onde deseja-se avaliar a função interpolada
x_new = np.linspace(0, lista_v[-1], 100)

# Avalia a função interpolada nos novos pontos
y_new = interp(x_new)

fig = go.Figure()

# Scatter plot dos dados originais
fig.add_trace(go.Scatter(x=lista_v, y=fracao, mode='markers', marker=dict(color='blue', symbol='circle'), name='Dados Originais'))

# Linha de interpolação PCHIP
fig.add_trace(go.Scatter(x=x_new, y=y_new, mode='lines', name='Interpolação PCHIP'))

# Layout
fig.update_layout(title='Resultados da Interpolação PCHIP',
                  xaxis_title='f',
                  yaxis_title='Probabilidade Acumulada',
                  showlegend=True,
                  plot_bgcolor='rgba(0,0,0,0)') # Define a cor de fundo do plot

fig.show()

#### Cálculo de Erro

In [33]:
k = 1.7*10**4
n = 5*10**6

z_score = 1.96
erro_n = z_score/(2*(n**(1/2)))
erro_k = 1/k
print(erro_n, erro_k)

erro_total = erro_n + erro_k
erro_total*(10**2)

0.00043826932358995873 5.882352941176471e-05


0.04970928530017235